In [16]:
from IPython.display import display, Markdown

In [17]:
import dotenv
import os

dotenv.load_dotenv()


def _getOrThrow(key):
    value = os.getenv(key)
    if value is None:
        raise RuntimeError(f"Missing environment variable: {key}")
    return value


OPENAI_API_KEY = _getOrThrow("OPENAI_API_KEY")


In [18]:
import requests


class GptCompleter:
    def __init__(self, system_prompt: str) -> None:
        self._system_prompt = system_prompt

    def complete(self, prompt: str) -> str:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            json={
                "model": "gpt-4-0125-preview",
                "max_tokens": 4000,
                "temperature": 0,
                "top_p": 1,
                "frequency_penalty": 0,
                "presence_penalty": 0,
                "messages": [
                    {"role": "system", "content": self._system_prompt},
                    {"role": "user", "content": prompt},
                ],
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
        )
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"]


In [19]:
from pathlib import Path


SYSTEM_PROMPT_TEPMPLATE = """\
You are a programming assistant. 

When asked to update, please provide only the code that you want to add or change.


Source files are:
```
{filenames}
```

Source code:

{source_files}
"""


SOURCE_PART_TEMPLATE = """\
name: `{filename}`

contents:
```
{contents}
```
"""

INCLUDE_SUFFIXES = [".py", ".md"]
EXCLUDE_DIRS = [".git", "__pycache__", ".venv"]


def all_files_recursively(dir=".", exclude_dirs=EXCLUDE_DIRS):
    result = []
    for p in Path(dir).iterdir():
        if p.is_dir() and p.name in exclude_dirs:
            continue
        if p.is_file():
            result.append(p)
        else:
            result.extend(all_files_recursively(p, exclude_dirs))
    return result


def build_sys_prompt(dir="."):
    all_files = all_files_recursively(dir)
    filenames = [str(p.relative_to(dir)) for p in all_files]
    parts = []
    for p in all_files:
        if p.suffix in INCLUDE_SUFFIXES:
            try:
                contents = p.read_text() or "EMPTY FILE"
            except Exception as e:
                print(f"Error reading {p}: {e}")
                return
            parts.append(
                SOURCE_PART_TEMPLATE.format(
                    filename=p.relative_to(dir), contents=contents
                )
            )
    return SYSTEM_PROMPT_TEPMPLATE.format(
        filenames="\n".join(filenames), source_files="\n".join(parts)
    )



In [20]:
display(Markdown(build_sys_prompt()))

You are a programming assistant. 

When asked to update, please provide only the code that you want to add or change.


Source files are:
```
app/src/plugins/task_manager/.keep
app/src/plugins/pdf_reader/.keep
app/src/server/config.py
app/src/server/main.py
app/src/heads/tg_bot/.keep
app/src/heads/tg_acc/.keep
app/src/heads/whatsapp/.keep
LICENSE
requirements.txt
copilot.ipynb
pyproject.toml
README.md
.gitignore
.env
config.yml
.env_ex
```

Source code:

name: `app/src/server/config.py`

contents:
```
EMPTY FILE
```

name: `app/src/server/main.py`

contents:
```
EMPTY FILE
```

name: `README.md`

contents:
```
# tao-multibot
LLM chat-bot infrastructure service featuring multiple bots instances with custom plugins and frontends

```



In [23]:
QUERY = """\
This is a test.
"""
res = GptCompleter(build_sys_prompt()).complete(QUERY)
display(Markdown(res))

How can I assist you with your test?